In [1]:

import pandas as pd
from pyspark.sql.functions import struct, col
from pyspark.sql.pandas.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StringType, DoubleType

from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
from sparkstudy.libs.tools import create_random_data
%load_ext autoreload
%autoreload 2
%matplotlib inline

COLUMNS = ["name","age","salary"]

比较下开不开启arrow的区别

测试下来，感觉性能提升有点奇怪。有时候会快，有时候会慢。

In [2]:
def test_performance(session_factory:DemoSQLSessionFactory, n:int = 100000):
    data = create_random_data(n)
    spark_session = session_factory.build_session()
    df = spark_session.createDataFrame(data,COLUMNS).cache()
    df.toPandas().head(5)

In [3]:
session_factory_arrow = DemoSQLSessionFactory(name="with arraw")
session_factory_arrow.add_config("spark.sql.execution.arrow.pyspark.enabled","true")
%time test_performance(session_factory_arrow)

CPU times: user 2.59 s, sys: 39 ms, total: 2.63 s
Wall time: 10.1 s


常规的HelloWorld的example。
页面上面的第一个例子。本质就是生成一个新的dataframe
1. 在annotation上面列出的是新的dataframe的col和类型
2. 他会自动的把pd的转换成spark的
3. 函数应该会分批node执行。然后再汇总。因为我看到了。hello world的函数会被执行好几次

In [4]:
session_factory_arrow.add_config('spark.sql.execution.arrow.maxRecordsPerBatch',10)
spark = session_factory_arrow.build_session()
test_data = create_random_data(row_num=1000)
basic_df = spark.createDataFrame(test_data,COLUMNS)
basic_df.show()

+----+---+-------------------+
|name|age|             salary|
+----+---+-------------------+
|   P|559|0.16409891492104978|
|   J|244|0.14893765822564886|
|   T|825| 0.8465170954153429|
|   L|617|0.24441519974300752|
|   B|369|0.05997399126491576|
|   A|820| 0.6820136758134797|
|   W|710|0.33806890904676257|
|   Q|869| 0.5121709882107319|
|   E|234|0.11226645551188152|
|   I|169| 0.3030591606796864|
|   E|436| 0.7236067744870015|
|   T|522|0.37965515031735997|
|   N|525|0.21233611808070207|
|   C|353|0.04290133090471937|
|   C| 93|0.06170264673252168|
|   L|235|0.07863983381189388|
|   X| 51| 0.6094419787359339|
|   D|923|0.45009256026484823|
|   L|185|0.20790219324212023|
|   O| 78| 0.7171646379115556|
+----+---+-------------------+
only showing top 20 rows



In [5]:
@pandas_udf("total double")
def func(s1: pd.Series, s2: pd.Series) -> pd.DataFrame:
    print("execute")
    s3 = pd.DataFrame()
    s3['total'] = s1 + s2
    return s3
basic_df.select(func("age","salary").alias("result")).show()

+--------------------+
|              result|
+--------------------+
|  [559.164098914921]|
|[244.14893765822566]|
| [825.8465170954154]|
|  [617.244415199743]|
|[369.05997399126494]|
| [820.6820136758134]|
| [710.3380689090468]|
| [869.5121709882108]|
|[234.11226645551187]|
|[169.30305916067968]|
|  [436.723606774487]|
| [522.3796551503174]|
| [525.2123361180807]|
| [353.0429013309047]|
| [93.06170264673253]|
| [235.0786398338119]|
| [51.60944197873594]|
| [923.4500925602648]|
|[185.20790219324212]|
| [78.71716463791155]|
+--------------------+
only showing top 20 rows



主要是想要看看。select方法，不能不能接受一个List

In [6]:
def to_str_func(s1: pd.Series) -> pd.Series:
    return s1.astype(dtype=str)
to_str = pandas_udf(to_str_func, returnType=StringType())

age_c = to_str("age").alias("age")
salary_c = to_str("salary").alias("salary")
selects = [age_c,salary_c]
basic_df.select(selects).show()

+---+-------------------+
|age|             salary|
+---+-------------------+
|559|0.16409891492104978|
|244|0.14893765822564886|
|825| 0.8465170954153429|
|617|0.24441519974300752|
|369|0.05997399126491576|
|820| 0.6820136758134797|
|710|0.33806890904676257|
|869| 0.5121709882107319|
|234|0.11226645551188152|
|169| 0.3030591606796864|
|436| 0.7236067744870015|
|522|0.37965515031735997|
|525|0.21233611808070207|
|353|0.04290133090471937|
| 93|0.06170264673252168|
|235|0.07863983381189388|
| 51| 0.6094419787359339|
|923|0.45009256026484823|
|185|0.20790219324212023|
| 78| 0.7171646379115556|
+---+-------------------+
only showing top 20 rows



测试以下。如果参数是不定的行不行

简单的来书，
- 确定的column个数，用Series
- 不确定用dataframe
- iterator是类似用流

In [7]:
@pandas_udf("double")
def to_sum_func(data: pd.DataFrame) -> pd.Series:
    return data.age*data.salary
cols = [col("age"),col("salary")]
headers = struct(cols)
#my_sum = pandas_udf(to_sum_func, returnType=DoubleType())
basic_df.select(to_sum_func(headers).alias("result")).show()


+------------------+
|            result|
+------------------+
| 91.73129344086682|
| 36.34078860705832|
| 698.3766037176579|
|150.80417824143564|
|22.130402776753915|
| 559.2512141670534|
|240.02892542320143|
|445.07658875512607|
| 26.27035058978028|
| 51.21699815486701|
|315.49255367633265|
| 198.1799884656619|
|111.47646199236858|
|15.144169809365938|
| 5.738346146124517|
|18.480360945795063|
|31.081540915532628|
| 415.4354331244549|
|38.461905749792244|
|55.938841757101336|
+------------------+
only showing top 20 rows



能不能用于SQL

In [8]:
basic_df.createOrReplaceTempView("pandas_udf")
spark.udf.register("pandas_to_str", to_str)
spark.sql("select pandas_to_str(age) from pandas_udf").show()

+------------------+
|pandas_to_str(age)|
+------------------+
|               559|
|               244|
|               825|
|               617|
|               369|
|               820|
|               710|
|               869|
|               234|
|               169|
|               436|
|               522|
|               525|
|               353|
|                93|
|               235|
|                51|
|               923|
|               185|
|                78|
+------------------+
only showing top 20 rows



basic_df.createOrReplaceTempView("pandas_udf")
spark.udf.register("pandas_to_str", to_str)
spark.sql("select pandas_to_str(age) from pandas_udf").show()

`__call__`这个方法能不能用哪用

In [9]:
class PandasFunc:
    def __call__(self, data: pd.DataFrame)-> pd.Series:
         return data.age*data.salary

cols = [col("age"),col("salary")]
headers = struct(cols)
class_my_sum = pandas_udf(PandasFunc(), returnType=DoubleType())
basic_df.select(class_my_sum(headers).alias("result")).show()

+------------------+
|            result|
+------------------+
| 91.73129344086682|
| 36.34078860705832|
| 698.3766037176579|
|150.80417824143564|
|22.130402776753915|
| 559.2512141670534|
|240.02892542320143|
|445.07658875512607|
| 26.27035058978028|
| 51.21699815486701|
|315.49255367633265|
| 198.1799884656619|
|111.47646199236858|
|15.144169809365938|
| 5.738346146124517|
|18.480360945795063|
|31.081540915532628|
| 415.4354331244549|
|38.461905749792244|
|55.938841757101336|
+------------------+
only showing top 20 rows



返回多列的处理方法。

In [10]:
@pandas_udf("col1 double, col2 double")
def to_multi_return_func(data: pd.DataFrame) -> pd.DataFrame:
    print("execute")
    s3 = pd.DataFrame()
    s3['col1'] = data.age
    s3['col2'] = data.salary
    return s3
cols = [col("age"),col("salary")]
headers = struct(cols)
#my_sum = pandas_udf(to_sum_func, returnType=DoubleType())
multi_return_df = basic_df.withColumn("abc",to_multi_return_func(headers))
multi_return_df.select(col("age"),col("salary"),col("abc.col1"),col("abc.col2")).show()

+---+-------------------+-----+-------------------+
|age|             salary| col1|               col2|
+---+-------------------+-----+-------------------+
|559|0.16409891492104978|559.0|0.16409891492104978|
|244|0.14893765822564886|244.0|0.14893765822564886|
|825| 0.8465170954153429|825.0| 0.8465170954153429|
|617|0.24441519974300752|617.0|0.24441519974300752|
|369|0.05997399126491576|369.0|0.05997399126491576|
|820| 0.6820136758134797|820.0| 0.6820136758134797|
|710|0.33806890904676257|710.0|0.33806890904676257|
|869| 0.5121709882107319|869.0| 0.5121709882107319|
|234|0.11226645551188152|234.0|0.11226645551188152|
|169| 0.3030591606796864|169.0| 0.3030591606796864|
|436| 0.7236067744870015|436.0| 0.7236067744870015|
|522|0.37965515031735997|522.0|0.37965515031735997|
|525|0.21233611808070207|525.0|0.21233611808070207|
|353|0.04290133090471937|353.0|0.04290133090471937|
| 93|0.06170264673252168| 93.0|0.06170264673252168|
|235|0.07863983381189388|235.0|0.07863983381189388|
| 51| 0.6094

这里的目的，主要还是为了验证一下partitionBy的用法

[normalize pyspark dataframe by group](https://stackoverflow.com/questions/54112439/normalize-pyspark-data-frame-by-group)

In [34]:
partition_key_data = [('A',x) for x in range(10)]+[('B',x) for x in range(10,20)]
partition_df = spark.createDataFrame(partition_key_data,["name","value"]).cache()
partition_df.show()

+----+-----+
|name|value|
+----+-----+
|   A|    0|
|   A|    1|
|   A|    2|
|   A|    3|
|   A|    4|
|   A|    5|
|   A|    6|
|   A|    7|
|   A|    8|
|   A|    9|
|   B|   10|
|   B|   11|
|   B|   12|
|   B|   13|
|   B|   14|
|   B|   15|
|   B|   16|
|   B|   17|
|   B|   18|
|   B|   19|
+----+-----+



In [43]:
@pandas_udf("new double,name string,value double",functionType=PandasUDFType.GROUPED_MAP)
def group_by_normalize(data) -> pd.DataFrame:
    value = data["value"]
    df = (value - value.mean())/value.std()
    data['new'] = df
    return data

partition_df.groupby("name").apply(group_by_normalize).show()

/Users/chandlersong/oneDrive/code/pyspark/venv/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


+--------------------+----+-----+
|                 new|name|value|
+--------------------+----+-----+
| -1.4863010829205867|   B| 10.0|
| -1.1560119533826787|   B| 11.0|
| -0.8257228238447705|   B| 12.0|
|-0.49543369430686224|   B| 13.0|
| -0.1651445647689541|   B| 14.0|
|  0.1651445647689541|   B| 15.0|
| 0.49543369430686224|   B| 16.0|
|  0.8257228238447705|   B| 17.0|
|  1.1560119533826787|   B| 18.0|
|  1.4863010829205867|   B| 19.0|
| -1.4863010829205867|   A|  0.0|
| -1.1560119533826787|   A|  1.0|
| -0.8257228238447705|   A|  2.0|
|-0.49543369430686224|   A|  3.0|
| -0.1651445647689541|   A|  4.0|
|  0.1651445647689541|   A|  5.0|
| 0.49543369430686224|   A|  6.0|
|  0.8257228238447705|   A|  7.0|
|  1.1560119533826787|   A|  8.0|
|  1.4863010829205867|   A|  9.0|
+--------------------+----+-----+

